In [2]:
!pip3 install gpt-2-simple

In [2]:
import gpt_2_simple as gpt2
import os
import requests

In [3]:
model_name = "345M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(
        model_name=model_name
    )  # model is saved into current directory under /models/124M/

In [ ]:
# file_name = "SQuAD_csv.txt"
# if not os.path.isfile(file_name):
# 	url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
# 	data = requests.get(url)

# 	with open(file_name, 'w') as f:
# 		f.write(data.text)

In [7]:
import pandas

# colnames = ['index', 'text', 'questions', 'answers']
data = pandas.read_csv(
    "SQuAD_csv.csv",
)

In [8]:
dataset = ""

text = data.context[:1000].tolist()
questions = data.question[:1000].tolist()
answers = data.text[:1000].tolist()


c, q, a, e = (
    "<|startoftext|>\n[CONTEXT]: ",
    "\n[QUESTION]:",
    "\n[ANSWER]:",
    "\n<|endoftext|>\n",
)

index = 0
while index < len(text):
    if index == 0 or text[index] != text[index - 1]:
        dataset += c + text[index]

    questionS = "".join(map(str, questions[index]))
    answerS = "".join(map(str, answers[index]))

    answerI = answerS
    # answerI = answerI[9 : : ]
    # print(answerI)
    dataset += q + questionS
    dataset += a + answerI

    if index == len(text) - 1 or text[index] != text[index + 1]:
        dataset += e
    index += 1
# print(dataset)
# print(dataset)

In [9]:
with open("QAdataset.txt", "w", encoding="utf-8") as f:
    f.write(dataset)
    f.close()

In [5]:
file_name = "QAdataset.txt"
sess = gpt2.start_tf_sess()

In [14]:
gpt2.finetune(
    sess,
    dataset=file_name,
    model_name="345M",
    steps=4000,
    restore_from="fresh",
    run_name="QA",
    print_every=1,
    sample_every=2000,
    save_every=100,
)

For larger models, the recommended finetune() parameters are:
	use_memory_saving_gradients = True
	only_train_transformer_layers = True
	accumulate_gradients = 1

Loading checkpoint models\345M\model.ckpt
INFO:tensorflow:Restoring parameters from models\345M\model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  2.22it/s]


dataset has 43534 tokens
Training...
[1 | 258.06] loss=1.92 avg=1.92
[2 | 492.09] loss=1.85 avg=1.89
[3 | 728.40] loss=1.78 avg=1.85
[4 | 957.67] loss=1.79 avg=1.84
[5 | 1181.28] loss=1.72 avg=1.81
[6 | 1406.44] loss=1.77 avg=1.80
[7 | 1626.95] loss=1.60 avg=1.77
[8 | 1846.36] loss=1.47 avg=1.74
[9 | 2070.99] loss=1.50 avg=1.71
[10 | 2291.48] loss=1.46 avg=1.68
[11 | 2508.62] loss=1.58 avg=1.67
[12 | 2725.58] loss=1.51 avg=1.66
[13 | 2943.38] loss=1.64 avg=1.66
interrupted
Saving checkpoint\QA\model-13


In [11]:
gpt2.load_gpt2(
    sess,
    checkpoint="latest",
    run_name="QA",
    checkpoint_dir="checkpoint",
    model_name=None,
    model_dir="models",
    multi_gpu=False,
    reuse=True,
)

Loading checkpoint checkpoint\QA\model-13
INFO:tensorflow:Restoring parameters from checkpoint\QA\model-13


In [13]:
ctx = input("Context: ")
qst = input("Question: ")
ans = "[ANSWER]:"
pre = "<|startoftext|>\n[CONTEXT]: " + ctx + "\n[QUESTION]:" + qst + "\n" + ans

In [14]:
gpt2.generate(
    sess,
    length=5,
    temperature=1,
    prefix=pre,
    nsamples=10,
    batch_size=10,
    run_name="QA",
)

<|startoftext|>
[CONTEXT]: Beyoncé Giselle Knowles was born in Houston, Texas, to Celestine Ann ""Tina"" Knowles (née Beyincé), a hairdresser and salon owner, and Mathew Knowles, a Xerox sales manager. Beyoncé's name is a tribute to her mother's maiden name. Beyoncé's younger sister Solange is also a singer and a former member of Destiny's Child. Mathew is African-American, while Tina is of Louisiana Creole descent (with African, Native American, French, Cajun, and distant Irish and Spanish ancestry). Through her mother, Beyoncé is a descendant of Acadian leader Joseph Broussard. She was raised in a Methodist household.
[QUESTION]:Where was Beyoncé born?
[ANSWER]:Houston
[QUESTION
<|startoftext|>
[CONTEXT]: Beyoncé Giselle Knowles was born in Houston, Texas, to Celestine Ann ""Tina"" Knowles (née Beyincé), a hairdresser and salon owner, and Mathew Knowles, a Xerox sales manager. Beyoncé's name is a tribute to her mother's maiden name. Beyoncé's younger sister Solange is also a singer a